In [13]:
cd ../src/

/Users/brent/Archiver/Workspace/qcqp/src


In [14]:
import collections

import numpy as np
import pandas as pd

from pyqptest.helpers import *

In [15]:
parser.print_help()
args, unknown = parser.parse_known_args()
params = BCParams()
kwargs, r_methods = params.produce_args(parser, METHOD_CODES)

usage: QCQP runner [-h] [--dump_instance DUMP_INSTANCE] [--r R]
                   [--fpath FPATH] [--n N] [--m M] [--pc PC]
                   [--time_limit TIME_LIMIT] [--verbose VERBOSE] [--bg BG]
                   [--bg_rd BG_RD]

optional arguments:
  -h, --help            show this help message and exit
  --dump_instance DUMP_INSTANCE
                        if save instance
  --r R                 {1: 'grb', 2: 'shor', 3: 'dshor', 4: 'msc', 5: 'emsc',
                        6: 'ssdp', 7: 'bb', 8: 'bb_msc', 9: 'shor_homo'}
  --fpath FPATH         path of the instance
  --n N                 dim of x
  --m M                 if randomly generated num of constraints
  --pc PC               if randomly generated problem branch_type
  --time_limit TIME_LIMIT
                        time limit of running.
  --verbose VERBOSE     if verbose
  --bg BG               backend used
  --bg_rd BG_RD         backend used, rank reduction
usage: QCQP runner [-h] [--dump_instance DUMP_INSTANCE] 

In [40]:
# r_methods = {'bb', 'grb', 'shor'}
r_methods = {'grb'}

In [41]:
n, m = 50, 0
# problem
problem_id = f"{n}:{m}:{0}"
# start
# qp = QPI.block(n, m, r=2, eps=0.5)
qp = QPI.normal(int(n), int(m), rho=0.4)
bd = Bounds(xlb=np.zeros(shape=(n, 1)), xub=np.ones(shape=(n, 1)))

# benchmarking

In [42]:
evals = []
results = {}
# run methods
for k in r_methods:
  func = METHODS[k]
  qp1 = bb_msc.QP(*qp.unpack())
  qp1.decompose()
  r = func(qp1, bd, params=params)
  reval = r.eval(problem_id)
  evals.append({**reval.__dict__, "method": k})
  results[k] = r
for k, r in results.items():
  print(f"{k} benchmark @{r.relax_obj}")
  r.check(qp)

Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter TimeLimit to 60.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter Threads to 1
   Prev: 0  Min: 0  Max: 1024  Default: 0
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 1 threads
Optimize a model with 0 rows, 50 columns and 0 nonzeros
Model fingerprint: 0x1dc4bd2a
Model has 713 quadratic objective terms
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 4e+00]
  QObjective range [2e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]

Continuous model is non-convex -- solving as a MIP.

Found heuristic solution: objective -0.0000000
Presolve time: 0.00s
Presolved: 970 rows, 755 columns, 3069 nonzeros
Presolved model has 11 quadratic constraint(s)
Pr

In [35]:
results['grb'].xval, results['bb'].xval.round(2)

(array([[ 0.75],
        [ 1.  ],
        [ 0.  ],
        [ 0.  ],
        [-0.  ],
        [ 1.  ],
        [-0.  ],
        [ 0.  ],
        [ 1.  ],
        [-0.  ]]),
 array([[0.75],
        [1.  ],
        [0.  ],
        [0.  ],
        [0.  ],
        [1.  ],
        [0.  ],
        [0.  ],
        [1.  ],
        [0.  ]]))

In [43]:
import pyqp.bg_msk_admm as pyadm
import importlib
importlib.reload(pyadm)
qp1 = bb_msc.QP(*qp.unpack())
qp1.decompose(decompose_method='eig-type2')
# initialize
m, n, dim = qp1.a.shape
xval = np.ones((n, dim))
sval = (xval.T @ xval).trace()
rho = 1
kappa = 0
mu = 0
xival = xval

In [44]:
# test run
max_iter = 100
_iter = 0
while True:
  _iter += 1
  r = pyadm.msc_subproblem_x(
    sval, xival, kappa, mu, rho, qp1, bd, solve=False, verbose=False
  )
  r.solve()
  xval = r.xval
  tval = r.tvar.level()[0]
  r_xi = pyadm.msc_subproblem_xi(
    xval, tval, kappa, mu, rho, qp1, bd, solve=False, verbose=False
  )
  r_xi.problem.solve()
  sval = r_xi.svar.level()[0]
  xival = r_xi.xivar.level().reshape(r_xi.xivar.getShape())
  residual_ts = tval - sval
  residual_xix = (xival * xval).sum() - tval
  print(
    f"iteration @{_iter} ## alm: {r.problem.primalObjValue(): .4f} norm t - s: {residual_ts: .4e}, 𝜉x - t: {residual_xix: .4e}"
  )
  if max(abs(residual_ts), abs(residual_xix)) < 1e-4:
    break
  kappa += residual_ts * rho
  mu += residual_xix * rho
  if _iter >= max_iter:
    break

iteration @1 ## alm: -77.0937 norm t - s: -1.1407e+00, 𝜉x - t: -1.1861e+01
iteration @2 ## alm: -117.8659 norm t - s: -3.6610e+00, 𝜉x - t: -2.4925e+00
iteration @3 ## alm: -159.6310 norm t - s: -1.5314e+00, 𝜉x - t: -2.3248e+00
iteration @4 ## alm: -180.1054 norm t - s: -9.7509e-01, 𝜉x - t: -2.2281e+00
iteration @5 ## alm: -183.2715 norm t - s: -7.4940e-01, 𝜉x - t: -2.3304e+00
iteration @6 ## alm: -180.5201 norm t - s: -6.4205e-01, 𝜉x - t: -2.2325e+00
iteration @7 ## alm: -177.9306 norm t - s: -6.2393e-01, 𝜉x - t: -1.9709e+00
iteration @8 ## alm: -177.1123 norm t - s: -6.6393e-01, 𝜉x - t: -1.6122e+00
iteration @9 ## alm: -177.5685 norm t - s: -5.7370e-01, 𝜉x - t: -1.4152e+00
iteration @10 ## alm: -177.0587 norm t - s: -6.3407e-01, 𝜉x - t: -1.1682e+00
iteration @11 ## alm: -176.8766 norm t - s: -6.5085e-01, 𝜉x - t: -9.9487e-01
iteration @12 ## alm: -175.9071 norm t - s: -5.4421e-01, 𝜉x - t: -9.7635e-01
iteration @13 ## alm: -174.8315 norm t - s: -4.7471e-01, 𝜉x - t: -9.4183e-01
iteration

In [45]:
r.xval, r.problem.primalObjValue()

(array([[ 1.    ],
        [ 1.    ],
        [-0.    ],
        [-0.    ],
        [ 1.    ],
        [ 1.    ],
        [ 1.    ],
        [ 0.    ],
        [-0.    ],
        [ 1.    ],
        [-0.    ],
        [ 0.9404],
        [ 1.    ],
        [-0.    ],
        [ 0.    ],
        [-0.    ],
        [ 1.    ],
        [ 0.    ],
        [ 0.    ],
        [ 1.    ],
        [ 1.    ],
        [ 1.    ],
        [-0.    ],
        [ 1.    ],
        [-0.    ],
        [-0.    ],
        [ 0.    ],
        [ 1.    ],
        [-0.    ],
        [-0.    ],
        [ 1.    ],
        [-0.    ],
        [ 0.    ],
        [-0.    ],
        [-0.    ],
        [ 1.    ],
        [ 0.    ],
        [ 0.    ],
        [ 1.    ],
        [ 0.9979],
        [-0.    ],
        [-0.    ],
        [ 0.    ],
        [-0.    ],
        [-0.    ],
        [ 1.    ],
        [-0.    ],
        [ 1.    ],
        [ 1.    ],
        [-0.    ]]),
 -177.04478547486394)

In [46]:
results['grb'].xval - r.xval

array([[ 0.    ],
       [ 0.    ],
       [ 0.    ],
       [ 0.    ],
       [ 0.    ],
       [ 0.    ],
       [-1.    ],
       [ 0.    ],
       [ 0.    ],
       [ 0.    ],
       [ 0.    ],
       [ 0.0596],
       [ 0.    ],
       [ 0.    ],
       [ 0.    ],
       [ 0.    ],
       [ 0.    ],
       [ 0.    ],
       [ 0.    ],
       [ 0.    ],
       [ 0.    ],
       [ 0.    ],
       [ 0.    ],
       [ 0.    ],
       [ 0.    ],
       [ 0.    ],
       [ 0.    ],
       [ 0.    ],
       [ 0.    ],
       [ 0.    ],
       [ 0.    ],
       [ 0.    ],
       [ 0.    ],
       [ 0.    ],
       [ 0.    ],
       [ 0.    ],
       [ 1.    ],
       [-0.    ],
       [ 0.    ],
       [ 0.0021],
       [ 0.    ],
       [ 0.    ],
       [ 0.0836],
       [ 0.    ],
       [ 0.    ],
       [ 0.    ],
       [ 0.    ],
       [ 0.    ],
       [ 0.    ],
       [ 0.    ]])